Check for dimensions of images

In [61]:
from scipy import ndimage, misc
from shutil import copy2
from skimage.color import rgb2gray
import skimage
import tensorflow as tf
import numpy as np
import math

def check_dimensions(directory, size=(110, 90)):
    for filename in os.listdir(directory):
        img = ndimage.imread(str(directory + '/'+ filename), mode='RGB')
        shape = img.shape
        print(shape)
        if shape[0] != size[0] or shape[1] != size[1]:
            print("{} out of shape".format(filename))

# check_dimensions('./utk_faces_renamed')

def flip_images(directory):
    baseFilename = "face_{}_{}.jpg"    
    if not os.path.exists("./utk_faces_extended"):
        os.mkdir("./utk_faces_extended")
        
    classCounter = img_per_class(directory)
    for filename in os.listdir(directory):
        imgClass = int(filename.split("_")[1])
        filePath = str(directory + '/' + filename)
        classCounter[imgClass] += 1
        
        img = ndimage.imread(filePath, mode='RGB')
        imgFlip = np.fliplr(img)
        
        newName = baseFilename.format(str(imgClass).zfill(3), str(classCounter[imgClass]).zfill(5))
        misc.imsave('./utk_faces_extended/{}'.format(newName), imgFlip)
        misc.imsave('./utk_faces_extended/{}'.format(filename), img)
            
def img_per_class(directory):
    ageRangeCounter = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}
    for filename in os.listdir(directory):
    # To count images per group when renamed
        group = int(filename.split("_")[1])
        ageRangeCounter[group] += 1
    
    totalSum = sum([v for k,v in ageRangeCounter.items()])
    print("Total sum: ", totalSum)
    print("Per class: ", ageRangeCounter)
    
    return ageRangeCounter

def class_percentile(ageRangeCounter, testing = 0.1, train = 0.9, verbose = False):  
    agePercentile = {1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: []}
    for i in range(1, 8):
        trainingQty = math.floor(ageRangeCounter[i] * train)
        testQty = ageRangeCounter[i] - trainingQty
        agePercentile[i] = [trainingQty , testQty]

    testCount = sum([v[1] for k,v in agePercentile.items()])
    trainCount = sum([v[0] for k,v in agePercentile.items()])
    if verbose:
        print("Hold out {}/{}  class count: {}".format(train * 100, testing * 100, agePercentile))
        print("Train count: ", trainCount)
        print("Test count: ", testCount)
    return agePercentile

def add_tarea3_train(tarea3Folder,folderTrain):
    baseName = "face_{}_{}.png"
    newBaseName = "face_{}_{}.jpg"
    imgCounter = img_per_class(folderTrain)
    for i in range(1, 8):
        for j in range(1,201):
            imgCounter[i] += 1
            srcPath = tarea3Folder + '/' + baseName.format(str(i).zfill(3), str(j).zfill(5))
            dstPath = folderTrain + '/' + newBaseName.format(str(i).zfill(3), str(imgCounter[i]).zfill(5))
            copy2(srcPath, dstPath)
            
def add_tarea3_test(tarea3Folder, folderTest):
    baseName = "face_{}_{}.png"
    newBaseName = "face_{}_{}.jpg"
    imgCounter = img_per_class(folderTest)
    for i in range(1, 8):
        for j in range(201, 241):
            imgCounter[i] += 1
            srcPath = tarea3Folder + '/' + baseName.format(str(i).zfill(3), str(j).zfill(5))
            dstPath = folderTest + '/' + newBaseName.format(str(i).zfill(3), str(imgCounter[i]).zfill(5))
            copy2(srcPath, dstPath)

def copy_img_to_folder(srcFolder, dstFolder):
    baseName = "face_{}_{}.jpg"
    
    if not os.path.exists(dstFolder):
        os.mkdir(dstFolder)
        
    imgCounter = img_per_class(dstFolder)

    for filename in os.listdir(srcFolder):
        imgClass = int(filename.split("_")[1])
        imgCounter[imgClass] += 1
        srcPath = srcFolder + '/' + filename
        dstPath = dstFolder + '/' + baseName.format(str(imgClass).zfill(3), str(imgCounter[imgClass]).zfill(5)) 
        copy2(srcPath, dstPath)

def resize_images(srcFolder, size=(110, 90)):
    for filename in os.listdir(srcFolder):
        img = ndimage.imread(srcFolder + '/' + filename, mode='RGB')
        shape = img.shape
        if shape[0] != size[0] or shape[1] != size[1]:
            imgResized = skimage.transform.resize(img, size)
            misc.imsave('./{}/{}'.format(srcFolder, filename), imgResized)

def to_gray(srcFolder):
    for filename in os.listdir(srcFolder):
        img = ndimage.imread(srcFolder + '/' + filename)
        imgGray = rgb2gray(img)
        misc.imsave(srcFolder + '/' + filename, imgGray)

# copy_img_to_folder('./age_db_cropped/', './datasets/train_folder')
# add_tarea3_train('./tarea3_fotos', './train_folder')
ageCounter = img_per_class('./datasets/train_folder')
ageCounter2 = img_per_class('./datasets/test_folder')
to_gray('./datasets/train_folder')
# resize_images('./datasets/train_folder')
# flip_images('./utk_faces_renamed/')

Total sum:  36676
Per class:  {1: 2425, 2: 3914, 3: 1658, 4: 5884, 5: 11844, 6: 8471, 7: 2480}
Total sum:  280
Per class:  {1: 40, 2: 40, 3: 40, 4: 40, 5: 40, 6: 40, 7: 40}
